In [ ]:
# Copyright 2020 Erik Härkönen. All rights reserved.
# This file is licensed to you under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License. You may obtain a copy
# of the License at http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software distributed under
# the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR REPRESENTATIONS
# OF ANY KIND, either express or implied. See the License for the specific language
# governing permissions and limitations under the License.

# Recreate StyleGAN1 style mixing image grid
from IPython.display import Image as IPyImage
from IPython.core.display import HTML 
#IPyImage('style_mixing.png')

In [ ]:
%matplotlib inline
from notebook_init import *

layer_names = [f'generator.layers.{i}' for i in range(14)] # annotate all shapes
inst = get_instrumented_model('BigGAN-512', 'promontory', layer_names, device)
model = inst.model
inst.close()

torch.manual_seed(0)
np.random.seed(0)

makedirs('out', exist_ok=True)

In [ ]:
def generate(trunc, cls, custom_seeds=[], layers=[0, 2, 4], N=5):
    inst.remove_edits()
    model.set_output_class(cls)
    
    custom_seeds = custom_seeds[:N] # limit to N images
    seeds = np.random.randint(np.iinfo(np.int32).max, size=N)
    seeds[:len(custom_seeds)] = custom_seeds
    print(seeds, trunc, cls)
    
    latents = [model.sample_latent(1, truncation=trunc, seed=s) for s in seeds]
    latent_a = latents[0]
    out_a = model.sample_np(latent_a)

    outputs = [model.sample_np(z) for z in latents]
    empty = np.ones_like(outputs[0])

    # Inputs B
    row0 = np.hstack([empty] + outputs[1:])
    rows = [row0]

    # Mix style starting from layer l
    for layer_num in layers:
        inst.close()
        layer_name = f'generator.layers.{layer_num}'
        inst.retain_layer(layer_name)

        imgs = []

        imgs.append(out_a)
        model.partial_forward(latent_a, layer_name)
        feat_a = inst.retained_features()[layer_name].detach()

        # Generate hybrids
        for i in range(1, len(latents)):
            # Use latent of B, early activations of A
            inst.edit_layer(layer_name, ablation=1.0, replacement=feat_a)
            out_b = model.sample_np(latents[i])
            imgs.append(out_b)

        rows.append(np.hstack(imgs))

    grid = np.vstack(rows)
    im = Image.fromarray((grid*255).astype(np.uint8))
    im.save(f'out/grid_{cls}.png')

    plt.figure(figsize=(15,15))
    plt.imshow(grid)
    plt.axis('off')
    plt.show()

    from IPython.display import Javascript, display
    
    if 0:
        display(Javascript("""
        require(
            ["base/js/dialog"], 
            function(dialog) {
                dialog.modal({
                    title: 'Debug',
                    body: 'Please close viewer window before continuing',
                    buttons: {
                        'Close': {}
                    }
                });
            }
        );
        """))
        im.show()
        

#generate(0.95, 'irish_setter', [716257571, 216337755, 602801999, 1027629257])
generate(0.95, 'barn', [237774802, 1498010115, 105741908, 857168362, 639216961])
#generate(0.95, 'coral_reef')
#generate(0.95, 'lighthouse', [1573600108])
#generate(0.95, 'seashore', [1891640828, 130794492, 1321047179, 750963629])
generate(0.95, 'castle', [995150904, 530702035])
#generate(0.95, 'golden_retriever', [])
#generate(0.95, 'goldfinch', [])
#generate(0.95, 'indigo_bunting', [1624898412])
#generate(0.95, 'red_wine', [])
#generate(0.95, 'anemone_fish', [11610217])
#generate(0.95, 'earthstar', [])
#generate(0.95, 'beer_bottle', [485603871, 527619953])
#generate(0.8, 'beer_glass', [])
#generate(0.95, 'church', [628962584, 1700971930]) # , 371570218, 1137007398, 1412786664


In [ ]:
# Show every layer for given content and style pair
def blend(cls, seed1, seed2):
    inst.remove_edits()
    model.set_output_class(cls)
    z1 = model.sample_latent(seed=seed1)
    z2 = model.sample_latent(seed=seed2)

    out1 = model.sample_np(z1)
    out2 = model.sample_np(z2)

    intermed = []
    for layer in range(0, 6, 1):
        inst.close()
        inst.remove_edits()
        layer_name = f'generator.layers.{layer}'
        inst.retain_layer(layer_name)

        # Content features up to layer
        model.partial_forward(z1, layer_name)
        feat = inst.retained_features()[layer_name].detach()

        # New style
        inst.edit_layer(layer_name, ablation=1.0, replacement=feat)
        intermed.append(model.sample_np(z2))

    imgs = np.hstack([out1] + intermed[::-1] + [out2])
    im = Image.fromarray((imgs*255).astype(np.uint8))
    im.save(f'out/{cls}_style_layer_comp.png')

    # Style blending by latent interpolation (does not keep geometry consistent)
    inst.remove_edits()
    lerp = lambda x,y,a : a*x+(1-a)*y
    imgs_latent_interp = []
    for a in np.linspace(0.0, 1.0, 8):
        z = lerp(z2, z1, a)
        imgs_latent_interp.append(model.sample_np(z))

    imgs_latent_interp = np.hstack(imgs_latent_interp)
    im = Image.fromarray((imgs_latent_interp*255).astype(np.uint8))
    im.save(f'out/{cls}_style_latent_comp.png')


blend('castle', 995150904, 1171165061)
blend('church', 628962584, 1700971930)
    